In [ ]:
import pandas as pd
import json
import math
from pathlib import Path
from PIL import Image, ImageDraw
import numpy as np
import matplotlib.pyplot as plt

# ----------------------------
# Config
# ----------------------------
ELEPHANTS_CSV = "/home/romeokienzler/Downloads/AED/test_elephants.csv"
IMAGES_CSV = "/home/romeokienzler/Downloads/AED/test_images.csv"
OUTPUT_JSON = "/home/romeokienzler/Downloads/AED/annotations_elephants_test.json"
IMAGES_DIR = Path("/home/romeokienzler/Downloads/AED/test_images")
SCALE_METERS = 6.0            # approximate elephant body length
ADJUST_BY_AGL = True          # scale by altitude above ground level

# ----------------------------
# Helper: compute box size
# ----------------------------
def compute_box_pixels(row, ref_agl, scale_meters=SCALE_METERS, adjust_by_agl=ADJUST_BY_AGL):
    gsd = float(row.get("GSD", float("nan")))
    agl = float(row.get("measured_altitude", float("nan"))) - float(row.get("terrain_altitude", 0.0))

    # Base size: meters / meters_per_pixel = pixels
    size_px = scale_meters / gsd if (gsd and gsd > 0) else None

    if adjust_by_agl and size_px is not None and agl > 0 and ref_agl > 0:
        size_px = size_px * (ref_agl / agl)

    if size_px is None or size_px <= 0:
        size_px = 50.0  # fallback

    return max(8.0, size_px)

# ----------------------------
# Build COCO JSON
# ----------------------------
def elephants_to_coco(elephants_csv, images_csv, output_json):
    df_ele = pd.read_csv(elephants_csv, header=None, names=["image_id", "x", "y"])  # cols: image_id, x, y
    #df_img = pd.read_csv(images_csv, header=None, names=["image_name", "image_width", "image_height", "GSD", "measured_altitude", "terrain_altitude"])     # cols: image_name, width, height, GSD, altitudes...

    df_img = pd.read_csv(
        images_csv,
        names=["image_name", "sortie_id", "image_width", "image_height", "GSD", "measured_altitude", "terrain_altitude", "gps_altitude"],
        dtype={
            "image_name": str,
            "sortie_id": str,
            "image_width": int,
            "image_height": int,
            "GSD": float,
            "measured_altitude": float,
            "terrain_altitude": float,
            "gps_altitude": float,
        }
    )


    print(df_img.iloc[0]['image_name'])
    df_ele["image_id"] = df_ele["image_id"].astype(str)
    df_img["image_name"] = df_img["image_name"].astype(str)


    merged = df_ele.merge(df_img, left_on="image_id", right_on="image_name", how="inner")

    # Reference AGL
    agl_series = merged["measured_altitude"] - merged["terrain_altitude"]
    ref_agl = float(agl_series.median()) if agl_series.notna().any() else 100.0

    images, annotations = [], []
    categories = [{"id": 1, "name": "elephant", "supercategory": "animal"}]

    # Map image names to IDs
    img_map = {}

    for i, row in df_img.iterrows():
        #img_map[row["image_name"]] = img_id
        images.append({
            "id": i,
            "file_name": f"{row['image_name']}.jpg",
            "width": int(row["image_width"]),
            "height": int(row["image_height"])
        })

    ann_id = 1
    for _, r in merged.iterrows():
        cx, cy = float(r["x"]), float(r["y"])
        img_w, img_h = int(r["image_width"]), int(r["image_height"])

        box_size = compute_box_pixels(r, ref_agl)
        half = box_size / 2
        x0 = max(0.0, cx - half)
        y0 = max(0.0, cy - half)
        x1 = min(img_w, cx + half)
        y1 = min(img_h, cy + half)
        w, h = x1 - x0, y1 - y0

        annotations.append({
            "id": ann_id,
            "image_id": r["image_name"],
            "category_id": 1,
            "bbox": [x0, y0, w, h],
            "area": w * h,
            "iscrowd": 0
        })
        ann_id += 1

    coco = {"images": images, "annotations": annotations, "categories": categories}
    with open(output_json, "w") as f:
        json.dump(coco, f, indent=2)
    print(f"Saved COCO annotations to {output_json}")
    return coco

# ----------------------------
# Visualization
# ----------------------------
def visualize(coco_json, images_dir=IMAGES_DIR, max_imgs=2):
    with open(coco_json, "r") as f:
        coco = json.load(f)

    img_by_id = {im["id"]: im for im in coco["images"]}
    anns_by_img = {}
    for ann in coco["annotations"]:
        anns_by_img.setdefault(ann["image_id"], []).append(ann)

    images_dir = Path(images_dir)

    for idx, img_info in enumerate(img_by_id.values()):
        if idx >= max_imgs:
            break
        path = images_dir / img_info["file_name"]
        if path.exists():
            im = Image.open(path).convert("RGB")
        else:
            im = Image.fromarray(np.full((img_info["height"], img_info["width"], 3), 255, np.uint8))
        draw = ImageDraw.Draw(im)
        for ann in anns_by_img.get(img_info["id"], []):
            x, y, w, h = ann["bbox"]
            draw.rectangle([x, y, x + w, y + h], outline="red", width=3)
        plt.figure(figsize=(8,6))
        plt.imshow(im)
        plt.axis("off")
    plt.show()

# ----------------------------
# Run
# ----------------------------
coco = elephants_to_coco(ELEPHANTS_CSV, IMAGES_CSV, OUTPUT_JSON)
#visualize(OUTPUT_JSON)
